# Content
1. Pendahuluan
2. Teori algoritma back-propagation kesalahan
3. Deskripsi database
4. Implementasi Jaringan Saraf Back-propagation
5. Kesimpulan
***
## 1. Pendahuluan

Jaringan Saraf Back-Propagation adalah jaringan feed-forward dengan arsitektur yang cukup sederhana. Arsitektur jaringan terdiri dari lapisan input, satu atau lebih lapisan tersembunyi, dan lapisan output. Jenis jaringan ini dapat membedakan data yang tidak dapat dipisahkan secara linear. Kami menggunakan algoritma error back-propagation untuk menyetel jaringan secara iteratif.
***
 ## 2. Teori algoritma back-propagation kesalahan

Algoritma back-propagation kesalahan terdiri dari dua langkah besar:
1. Meneruskan makanan dari input dari database ke lapisan input kemudian ke lapisan tersembunyi dan akhirnya ke lapisan output.
2. Menghitung kesalahan output dan memberi makan mundur untuk menyetel variabel jaringan.
***
## 3. Deskripsi database

Dalam contoh ini, kami akan menggunakan basis data Kanker Payudara Duke yang terdiri dari [86] entri dan [7129] atribut ditambah atribut kelas yang terletak di kolom pertama. Data ini bersifat numerik dan tidak memiliki nilai yang hilang.
***
## 4. Back-propagation Neural Network implementation

Pertama-tama, kita perlu memuat basis data.

In [31]:
import numpy as np 
from sklearn.model_selection import train_test_split #library train_test_split untuk membagi data training dan data testing

db = np.loadtxt("duke-breast-cancer.txt")
print("Database raw shape (%s,%s)" % np.shape(db))

Database raw shape (86,7130)


Sekarang kita harus mengacaknya dan kemudian membaginya menjadi 90% untuk pelatihan dan 10% untuk pengujian agar jaringan dapat melatih dirinya sendiri lebih baik. Jika diperlukan, Anda juga dapat menormalkan basis data tersebut.

In [32]:
np.random.shuffle(db) 
y = db[:, 0] #mengambil kolom pertama dari database sebagai target
x = np.delete(db, [0], axis=1) #menghapus kolom pertama dari database sebagai input
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1) 
print(np.shape(x_train),np.shape(x_test)) 

(77, 7129) (9, 7129)


Sekarang kita harus membuat vektor lapisan tersembunyi, matriks bobot, vektor lapisan output, dan matriks bobot tersembunyi. Kami memilih lapisan tersembunyi terdiri dari sejumlah [72] perceptron tersembunyi. Lapisan output perlu memiliki sejumlah perceptron yang sama dengan jumlah kelas. Matriks bobot akan memiliki bentuk berikut: baris = jumlah atribut basis data, kolom = jumlah perceptron lapisan tersembunyi, dan matriks bobot tersembunyi akan memiliki bentuk berikut: baris = panjang lapisan tersembunyi, kolom = jumlah perceptron lapisan output.

In [33]:
hidden_layer = np.zeros(72) #inisialisasi hidden layer dengan 72 neuron
weights = np.random.random((len(x[0]), 72)) #inisialisasi bobot dengan nilai random
output_layer = np.zeros(2) 
hidden_weights = np.random.random((72, 2))

Untuk melanjutkan, kita perlu mengimplementasikan beberapa fungsi penting:

1. Fungsi penjumlahan (Sum function)
2. Fungsi aktivasi (Activation function)
3. Fungsi SoftMax (SoftMax function)
4. Fungsi Menghitung Ulang Bobot (Recalculate Weights function)
5. Fungsi Back-propagation (Back-propagation function)

## Sum function
![sum.png](https://image.ibb.co/i3EM27/sum.png)
- s_i = jumlah untuk perceptron ke-[i] dari lapisan tersebut.
- Xi = input
- Wi = bobot
- Wj = bias
- y = output


In [34]:
def sum_function(weights, index_locked_col, x):
    result = 0
    for i in range(0, len(x)): #perulangan untuk menghitung hasil perkalian antara input dan bobot
        result += x[i] * weights[i][index_locked_col]
    return result

## Activation function
![image.png](https://image.ibb.co/eTfYFS/g.png)
- \( g(s_i) \) adalah aktivasi dari perceptron ke-\(i\) pada lapisan.
- \( s_i \) adalah hasil dari fungsi jumlah untuk perceptron ke-\(i\) pada lapisan, yang dihitung menggunakan bobot dan input.
- \( \tanh \) adalah fungsi tangen hiperbolik, yang menghasilkan nilai di rentang \((-1, 1)\).
- \( 1.7159 \) adalah konstanta untuk mengubah rentang fungsi tangen hiperbolik menjadi \((-1.7159, 1.7159)\). Ini merupakan pendekatan yang sering digunakan untuk mempercepat konvergensi jaringan selama pelatihan.

In [35]:
def activate_layer(layer, weights, x): 
    for i in range(0, len(layer)): 
        layer[i] = 1.7159 * np.tanh(2.0 * sum_function(weights, i, x) / 3.0)

## SoftMax function

Fungsi softmax, atau fungsi eksponensial yang dinormalisasi, adalah generalisasi dari fungsi logistik yang "menyempitkan" vektor z berdimensi K dari nilai real sembarang menjadi vektor σ(z) berdimensi K dari nilai real dalam rentang (0, 1) yang jumlahnya adalah 1.
- ![image.png](softmaxx.png)


In [36]:
def soft_max(layer): 
    soft_max_output_layer = np.zeros(len(layer)) #inisialisasi output layer dengan nilai 0
    for i in range(0, len(layer)):
        denominator = 0 #denominator adalah nilai pembagi
        for j in range(0, len(layer)):
            denominator += np.exp(layer[j] - np.max(layer)) 
        soft_max_output_layer[i] = np.exp(layer[i] - np.max(layer)) / denominator #rumus softmax
    return soft_max_output_layer

## Recalculate weights function
Di sini kita menyetel bobot jaringan dan matriks bobot tersembunyi. Kita akan menggunakan ini di dalam fungsi backpropagation.
- ![image.png](https://image.ibb.co/moBepn/w.png)

In [37]:
def recalculate_weights(learning_rate, weights, gradient, activation):
    for i in range(0, len(weights)):
        for j in range(0, len(weights[i])):
            weights[i][j] = (learning_rate * gradient[j] * activation[i]) + weights[i][j]

## Back-propagation function
Dalam fungsi ini, kita mencari gradien lapisan output dan gradien lapisan tersembunyi untuk menghitung ulang bobot jaringan.
- Rumus gradien lapisan output:
- ![image.png](https://image.ibb.co/eJ9qUn/go.png)
- Hidden gradient formula
- ![image.png](https://image.ibb.co/mYQ3h7/gh.png)

In [38]:
def back_propagation(hidden_layer, output_layer, one_hot_encoding, learning_rate, x): #one_hot_encoding adalah target, learning_rate adalah laju pembelajaran
    output_derivative = np.zeros(2) #inisialisasi turunan output
    output_gradient = np.zeros(2) #inisialisasi gradien output
    for i in range(0, len(output_layer)): #perulangan untuk menghitung turunan output
        output_derivative[i] = (1.0 - output_layer[i]) * output_layer[i]
    for i in range(0, len(output_layer)): #perulangan untuk menghitung gradien output
        output_gradient[i] = output_derivative[i] * (one_hot_encoding[i] - output_layer[i])
    hidden_derivative = np.zeros(72) 
    hidden_gradient = np.zeros(72)
    for i in range(0, len(hidden_layer)): #perulangan untuk menghitung turunan hidden layer
        hidden_derivative[i] = (1.0 - hidden_layer[i]) * (1.0 + hidden_layer[i])
    for i in range(0, len(hidden_layer)): #perulangan untuk menghitung gradien hidden layer
        sum_ = 0 
        for j in range(0, len(output_gradient)): 
            sum_ += output_gradient[j] * hidden_weights[i][j]
        hidden_gradient[i] = sum_ * hidden_derivative[i]

    #menghitung ulang bobot hidden layer dan output layer
    recalculate_weights(learning_rate, hidden_weights, output_gradient, hidden_layer) 
    recalculate_weights(learning_rate, weights, hidden_gradient, x)

Next we can [one hot encode](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science) our output and start training our network iterative.

In [39]:
one_hot_encoding = np.zeros((2,2)) #untuk mengubah target menjadi one hot encoding
for i in range(0, len(one_hot_encoding)): 
    one_hot_encoding[i][i] = 1
training_correct_answers = 0
for i in range(0, len(x_train)): #perulangan untuk proses pembelajaran 
    activate_layer(hidden_layer, weights, x_train[i])
    activate_layer(output_layer, hidden_weights, hidden_layer)
    output_layer = soft_max(output_layer)
    training_correct_answers += 1 if y_train[i] == np.argmax(output_layer) else 0
    back_propagation(hidden_layer, output_layer, one_hot_encoding[int(y_train[i])], -1, x_train[i])
print("MLP Correct answers while learning: %s / %s (Accuracy = %s) on %s database." % (training_correct_answers, len(x_train),
                                                                                       training_correct_answers/len(x_train),"Duke breast cancer"))

MLP Correct answers while learning: 41 / 77 (Accuracy = 0.5324675324675324) on Duke breast cancer database.


Akurasi pengujian bergantung pada matriks bobot yang dihasilkan secara acak dan laju pembelajaran. Menggunakan laju pembelajaran dan bobot yang berbeda akan menghasilkan akurasi yang berbeda pula.

In [40]:
testing_correct_answers = 0
for i in range(0, len(x_test)):
    activate_layer(hidden_layer, weights, x_test[i])
    activate_layer(output_layer, hidden_weights, hidden_layer)
    output_layer = soft_max(output_layer)
    testing_correct_answers += 1 if y_test[i] == np.argmax(output_layer) else 0
print("MLP Correct answers while testing: %s / %s (Accuracy = %s) on %s database" % (testing_correct_answers, len(x_test),
                                                                                     testing_correct_answers/len(x_test), "Duke breast cancer"))

MLP Correct answers while testing: 9 / 9 (Accuracy = 1.0) on Duke breast cancer database


Pada set pengujian ini, akurasi dapat mencapai 100% bahkan dengan jumlah perceptron tersembunyi yang tepat dalam lapisan tersembunyi. Dalam contoh ini, kami menggunakan laju pembelajaran [-1] dengan total [72] perceptron tersembunyi dalam lapisan tersembunyi.


***
## 5.  Conclusion
Dalam tes ini, kami telah menunjukkan bahwa jaringan saraf back-propagation berkinerja baik pada kumpulan data yang besar. Kinerjanya dapat ditingkatkan dengan mengubah jumlah neuron tersembunyi dan laju pembelajaran. Karena pelatihannya yang iteratif dan berbasis gradien, kecepatan umumnya jauh lebih lambat dari yang dibutuhkan, sehingga membutuhkan waktu yang cukup lama untuk dilatih pada kumpulan data yang sangat besar. Kami tidak dapat mengatakan bahwa ada jaringan yang sempurna untuk setiap jenis basis data. Jadi teruslah menguji data Anda pada berbagai jaringan saraf dan lihat mana yang paling cocok.

Saya harap notebook ini membantu Anda memulai perjalanan Anda ke dunia pembelajaran mesin dan big data.